# Stock Market Analysis: Tesla & GameStop
**Project Author:** Malik Tahayneh

**Date:** December 2025

## Executive Summary
This project analyzes the historical stock performance and revenue data of two major companies: **Tesla (TSLA)** and **GameStop (GME)**. By utilizing automated data extraction techniques (API scraping with `yfinance` and HTML web scraping with `BeautifulSoup`), this analysis correlates share price trends with corporate revenue growth.

The goal is to build a dynamic dashboard that visualizes stock data against financial performance to identify key growth patterns.

## Methodology
1.  **Data Extraction:** Retrieving historical stock data via the `yfinance` API.
2.  **Web Scraping:** Parsing quarterly revenue reports from financial websites using `BeautifulSoup`.
3.  **Data Processing:** Cleaning and structuring data using `Pandas`.
4.  **Visualization:** Plotting historical trends using `Matplotlib` to derive insights.

### 1. Environment Setup and Library Import
Installing necessary libraries for data collection and visualization.

In [ ]:
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install matplotlib

In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### 2. Utility Function: Graph Generation
We define a reusable helper function, `make_graph`, which accepts stock data and revenue data to plot a dual-axis visualization of **Share Price** vs. **Revenue** over time.

In [ ]:
def make_graph(stock_data, revenue_data, stock):
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']

    fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

    # Stock price
    axes[0].plot(pd.to_datetime(stock_data_specific.Date), stock_data_specific.Close.astype("float"), label="Share Price", color="blue")
    axes[0].set_ylabel("Price ($US)")
    axes[0].set_title(f"{stock} - Historical Share Price")

    # Revenue
    axes[1].plot(pd.to_datetime(revenue_data_specific.Date), revenue_data_specific.Revenue.astype("float"), label="Revenue", color="green")
    axes[1].set_ylabel("Revenue ($US Millions)")
    axes[1].set_xlabel("Date")
    axes[1].set_title(f"{stock} - Historical Revenue")

    plt.tight_layout()
    plt.show()

### 3. Case Study: Tesla (TSLA)

#### 3.1 Data Extraction via API
We utilize the `yfinance` library to extract Tesla's historical stock data, setting the period to "max" to capture the full market history.

In [ ]:
stock = yf.Ticker("TSLA")
tesla_data = stock.history(period="max")
tesla_data.reset_index(inplace=True)
tesla_data.head()

#### 3.2 Web Scraping Revenue Data
To analyze financial health, we scrape Tesla's quarterly revenue data from Macrotrends. The raw HTML table is parsed and cleaned to remove currency symbols and format the dates.

In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html5lib")

In [ ]:
df = pd.read_html(html_data)
tesla_revenue = df[1]
tesla_revenue.columns = ["Date", "Revenue"]

# Cleaning Data: Remove dollar signs and commas
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\\$',"",regex=True)

# Removing nulls and empty strings
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

tesla_revenue.tail()

### 4. Case Study: GameStop (GME)

#### 4.1 Data Extraction via API
Similar to the Tesla analysis, we extract GameStop's historical market data to examine its volatility and long-term trends.

In [ ]:
ticker = yf.Ticker("GME")
gme_data = ticker.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()

#### 4.2 Web Scraping Revenue Data
We scrape GameStop's revenue reports to contrast its financial performance against its stock price movements.

In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data2 = requests.get(url).text
soup = BeautifulSoup(html_data2, "html.parser")

In [ ]:
df = soup.find_all("tbody")[1]
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in df.find_all("tr"):
    col = row.find_all("td")
    if col:
        date = col[0].text
        revenue = col[1].text
        gme_revenue = pd.concat([gme_revenue, pd.DataFrame({"Date": [date], "Revenue": [revenue]})], ignore_index=True)

# Cleaning Data
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\\$',"",regex=True)
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

gme_revenue.tail()

### 5. Strategic Analysis & Visualization

#### 5.1 Tesla Performance Dashboard
The graph below visualizes the correlation between Tesla's stock price and its reported revenue.
*(Note: Visualizing data up to June 2021)*

In [ ]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

#### 5.2 GameStop Performance Dashboard
The graph below highlights GameStop's revenue trends alongside its market valuation.

In [ ]:
make_graph(gme_data, gme_revenue, 'GameStop')

## Conclusion
This analysis successfully integrated API usage and Web Scraping to build a financial dashboard for Tesla and GameStop.

**Key Findings:**
* **Tesla:** Shows a strong positive correlation between revenue growth and stock price, indicative of a growth-stage company.
* **GameStop:** Displays high volatility, with stock price movements that often dissociate from traditional revenue trends, highlighting its nature as a "meme stock" during specific periods.

**Future Work:**
* Incorporate sentiment analysis from news sources to correlate public perception with stock movement.
* Automate the script to run weekly for real-time updates.